In [1]:
import ee
import geemap


In [2]:
m = geemap.Map()

In [3]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from IPython.display import display

# -----------------------------------------------------------------------------
# Initialization
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# Define region
# -----------------------------------------------------------------------------
def get_florida_geometry():
    return (ee.FeatureCollection("TIGER/2018/States")
            .filter(ee.Filter.eq('NAME', 'Florida'))
            .first()
            .geometry())

# -----------------------------------------------------------------------------
# Landsat parameter helper
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    if 1985 <= year <= 2011:
        coll_id = 'LANDSAT/LT05/C02/T1_L2'
        bands = {
            'LST': 'ST_B6',
            'NDVI': ('SR_B4', 'SR_B3'),
            'NDBI': ('SR_B5', 'SR_B4')
        }
    elif 2012 <= year <= 2013:
        coll_id = 'LANDSAT/LE07/C02/T1_L2'
        bands = {
            'LST': 'ST_B6',
            'NDVI': ('SR_B4', 'SR_B3'),
            'NDBI': ('SR_B5', 'SR_B4')
        }
    elif 2014 <= year <= 2022:
        coll_id = 'LANDSAT/LC08/C02/T1_L2'
        bands = {
            'LST': 'ST_B10',
            'NDVI': ('SR_B5', 'SR_B4'),
            'NDBI': ('SR_B6', 'SR_B5')
        }
    else:
        coll_id = 'LANDSAT/LC09/C02/T1_L2'
        bands = {
            'LST': 'ST_B10',
            'NDVI': ('SR_B5', 'SR_B4'),
            'NDBI': ('SR_B6', 'SR_B5')
        }
    return coll_id, bands[param]

# -----------------------------------------------------------------------------
# Masking function without client-side getInfo in map
# -----------------------------------------------------------------------------
def apply_mask(image, use_radsat=False):
    # Mask clouds & shadows using QA_PIXEL bits
    masked = image.updateMask(
        image.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    )
    # Optionally mask saturated pixels
    if use_radsat:
        masked = masked.updateMask(
            masked.select('QA_RADSAT').eq(0)
        )
    return masked

# -----------------------------------------------------------------------------
# Image processing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year):
    coll_id, band = get_landsat_params(year, 'LST')
    coll = (ee.ImageCollection(coll_id)
            .filterBounds(FLORIDA)
            .filterDate(f'{year}-01-01', f'{year}-12-31'))

    # Pull band list once to see if QA_RADSAT exists
    bands = coll.first().bandNames().getInfo()
    use_radsat = 'QA_RADSAT' in bands

    def to_celsius(img):
        return (img.select(band)
                   .multiply(0.00341802)
                   .add(149.0)
                   .subtract(273.15)
                   .rename('surface_temp'))

    return (coll
            .map(lambda img: apply_mask(img, use_radsat=use_radsat))
            .map(to_celsius)
            .median()
            .clip(FLORIDA)
            .selfMask())

def get_ndvi(year):
    coll_id, (nir, red) = get_landsat_params(year, 'NDVI')
    def compute_ndvi(img):
        return img.normalizedDifference([nir, red]).rename('NDVI')
    return (ee.ImageCollection(coll_id)
            .filterBounds(FLORIDA)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda img: apply_mask(img))
            .map(compute_ndvi)
            .median()
            .clip(FLORIDA))

def get_ndbi(year):
    coll_id, (swir, nir) = get_landsat_params(year, 'NDBI')
    def compute_ndbi(img):
        return img.normalizedDifference([swir, nir]).rename('NDBI')
    return (ee.ImageCollection(coll_id)
            .filterBounds(FLORIDA)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda img: apply_mask(img))
            .map(compute_ndbi)
            .median()
            .clip(FLORIDA))

# -----------------------------------------------------------------------------
# Legend and Map Update (unchanged from your version)
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    for c in m.controls:
        if getattr(c, 'is_legend', False):
            m.remove_control(c)
    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    if title == 'LULC':
        for v, c, l in zip(NLCD_VALUES, NLCD_COLORS, NLCD_LABELS):
            html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:#{c};margin-right:4px;border:1px solid #aaa;'></div>{v}: {l}</div>"
    else:
        palette = vis['palette']
        step = (vis['max'] - vis['min']) / len(palette)
        for i, col in enumerate(palette):
            lo, hi = vis['min'] + i*step, vis['min'] + (i+1)*step
            label = (
                f"≤ {hi:.2f}" if i == 0 else
                f"> {lo:.2f}" if i == len(palette)-1 else
                f"{lo:.2f} – {hi:.2f}"
            )
            html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:#{col};margin-right:4px;border:1px solid #aaa;'></div>{label}</div>"
    html += "</div>"
    widget = widgets.HTML(html)
    ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    ctrl.is_legend = True
    m.add_control(ctrl)

def update_map(*_):
    global _prev
    if _prev:
        m.remove_layer(_prev)

    p, y = param_dd.value, int(year_dd.value)
    info_lbl.value = f'Loading {p} {y}…'
    if p == 'LST':
        img, vis, name = get_annual_lst(y), thermal_vis, f'LST {y}'
    elif p == 'UTFVI':
        lst = get_annual_lst(y)
        mean = ee.Number(lst.reduceRegion(ee.Reducer.mean(), FLORIDA, 1000).get('surface_temp'))
        img, vis, name = lst.subtract(mean).divide(mean).rename('UTFVI'), utfvi_vis, f'UTFVI {y}'
    elif p == 'Hotspots':
        lst = get_annual_lst(y)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), True),
                                 FLORIDA, 1000)
        mean, std = ee.Number(stats.get('surface_temp_mean')), ee.Number(stats.get('surface_temp_stdDev'))
        img = lst.gt(mean.add(std.multiply(2))).rename('hotspot')
        vis, name = hotspot_vis, f'Hotspots {y}'
    elif p == 'NDVI':
        img, vis, name = get_ndvi(y), ndvi_vis, f'NDVI {y}'
    elif p == 'NDBI':
        img, vis, name = get_ndbi(y), ndbi_vis, f'NDBI {y}'
    else:
        img = NLCD_COLLECTION.filter(ee.Filter.eq('system:index', str(y))).first().select('b1').clip(FLORIDA)
        vis, name = {'min':11, 'max':95, 'palette':NLCD_COLORS}, f'LULC {y}'

    _prev = m.add_layer(img, vis, name)
    add_legend(vis, 'LULC' if p == 'LULC' else p)
    info_lbl.value = f'{p} {y} loaded'

def on_param_change(change):
    if change.new == 'LULC':
        year_dd.options = NLCD_YEARS
        year_dd.value = NLCD_YEARS[-1]
    else:
        year_dd.options = LST_YEARS
        year_dd.value = max(LST_YEARS)
    update_map()

# -----------------------------------------------------------------------------
# Main execution
# -----------------------------------------------------------------------------
initialize_earth_engine()
FLORIDA = get_florida_geometry()
LST_YEARS = list(range(1985, 2025))
NLCD_COLLECTION = ee.ImageCollection("projects/sat-io/open-datasets/USGS/ANNUAL_NLCD/LANDCOVER")
NLCD_YEARS = NLCD_COLLECTION.aggregate_array("system:index").distinct().sort().getInfo()

NLCD_VALUES = [11,12,21,22,23,24,31,41,42,43,52,71,81,82,90,95]
NLCD_COLORS = ['466b9f','d1def8','dec5c5','d99282','eb0000','ab0000',
               'b3ac9f','68ab5f','1c5f2c','b5c58f','ccb879','dfdfc2',
               'dcd939','ab6c28','b8d9eb','6c9fb8']
NLCD_LABELS = ['Open Water','Perennial Ice/Snow','Dev, Open Space','Dev, Low Intensity',
               'Dev, Medium Intensity','Dev, High Intensity','Barren Land',
               'Deciduous Forest','Evergreen Forest','Mixed Forest','Shrub/Scrub',
               'Grassland/Herbaceous','Pasture/Hay','Cultivated Crops',
               'Woody Wetlands','Emergent Herbaceous Wetlands']

thermal_vis = {'min': -3, 'max': 37,
               'palette': ['000080','0000FF','00FFFF','00FF00','FFFF00','FFA500','FF0000']}
utfvi_vis = {'min': -0.3, 'max': 0.3, 'palette': ['0000FF','FFFFFF','FF0000']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['00000000','FF0000']}
ndvi_vis = {'min': -0.2, 'max': 1,
            'palette': ['FFFFFF','CE7E45','FCD163','66A000','207401']}
ndbi_vis = {'min': -0.5, 'max': 0.5,
            'palette': ['FFFFFF','B4B4B4','999999','7F7F7F','666666','4C4C4C','333333']}

# Widgets & map
m = geemap.Map(center=[28.0, -82.4], zoom=7)
m.addLayer(FLORIDA, {'color': 'white', 'fillColor': '00000000'}, 'Florida')

param_dd = widgets.Dropdown(
    options=['LST','UTFVI','Hotspots','NDVI','NDBI','LULC'],
    value='LST',
    description='Parameter:'
)
year_dd = widgets.Dropdown(options=LST_YEARS, value=2024, description='Year:')
info_lbl = widgets.Label()
_prev = None

param_dd.observe(on_param_change, names='value')
year_dd.observe(update_map, names='value')

display(widgets.HBox([param_dd, year_dd]), info_lbl)
display(m)
update_map()


Label(value='')

Map(center=[28.0, -82.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

To Create Box Plots for NDVI, NDBI and LST:

In [4]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    """Authenticates and initializes the Earth Engine API."""
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info = STATE_FC.getInfo()['features']
STATE_NAMES = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40, 'palette': ['blue', 'green', 'yellow', 'red']}
ndvi_vis = {'min': 0, 'max': 1, 'palette': ['white', 'lightgreen', 'darkgreen']}
ndbi_vis = {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'blue']}
utfvi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'red']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['lightgray', 'red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    """Selects Landsat collection and bands based on the year."""
    if 1985 <= year <= 2011:
        coll = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2012 <= year <= 2013:
        coll = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2014 <= year <= 2022:
        coll = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    else:
        coll = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    """Applies cloud, shadow, and optional radiometric saturation masks."""
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    """Computes the annual median Land Surface Temperature in Celsius."""
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                       .multiply(0.00341802)
                       .add(149.0)
                       .subtract(273.15)
                       .rename('surface_temp'))
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i, True))
              .map(to_celsius)
              .median()
              .clip(region)
              .selfMask())

def get_ndvi(year, region):
    """Computes the annual median NDVI."""
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(apply_mask)
              .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
              .median()
              .clip(region))

def get_ndbi(year, region):
    """Computes the annual median NDBI."""
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(apply_mask)
              .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
              .median()
              .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    """Removes old legends and adds a new one to the map."""
    # remove old
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)
    # build new
    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    pal, st, mx = vis['palette'], vis['min'], vis['max']
    step = (mx - st) / len(pal)
    for i, col in enumerate(pal):
        lo, hi = st + i * step, st + (i + 1) * step
        lbl = f"≤{hi:.2f}" if i == 0 else (f">{lo:.2f}" if i == len(pal) - 1 else f"{lo:.2f}-{hi:.2f}")
        html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:{col};'></div>{lbl}</div>"
    html += "</div>"
    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    """Updates the county dropdown based on the selected state."""
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    county_dd.value = names[0]

# -----------------------------------------------------------------------------
# 9. Map update (clip to selected region)
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    """Calculates and displays the selected parameter on the map."""
    global _prev_layer, REGION
    if _prev_layer:
        m.remove_layer(_prev_layer)

    # choose feature
    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value

    # compute image & vis
    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION), ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            REGION, scale_dd.value
        )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    # add layer to map
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"Displaying: {p} {yr} (clipped to {region_type_dd.value})"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def download_map(b):
    """CORRECTED: Generates and downloads a PNG for the currently displayed map."""
    yr, p = int(year_dd.value), param_dd.value
    info_lbl.value = f"Preparing download for {p}..."
    
    # Re-calculate image and vis for download to ensure it's correct
    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION), ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), REGION, scale_dd.value)
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    url = img.getThumbURL({
        'min': vis['min'], 'max': vis['max'],
        'palette': vis['palette'],
        'region': REGION,
        'dimensions': resolution_dd.value,
        'format': 'png'
    })
    resp = requests.get(url)
    county_name = f"_{county_dd.value}" if region_type_dd.value == 'County' else ''
    fn = f"map_{p}_{state_dd.value}{county_name}_{yr}.png".replace(" ", "_")
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

def download_animation(b):
    """CORRECTED: Downloads a GIF animation with the year overlaid on each frame."""
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    p = param_dd.value
    # Note: Animation is only implemented for LST and NDVI for simplicity.
    if p not in ['LST', 'NDVI']:
        info_lbl.value = f"Animation not available for '{p}'. Please choose LST or NDVI."
        return

    info_lbl.value = "Preparing animation... This may take a moment."
    yrs = list(range(start, end + 1, step_dd.value))
    vis_params = thermal_vis if p == 'LST' else ndvi_vis
    frames = []

    for yr in yrs:
        # Get the base data image
        img = get_annual_lst(yr, REGION) if p == 'LST' else get_ndvi(yr, REGION)
        
        # Create a text image for the year label
        text_image = ee.Image().paint(REGION, 0, 2).rename('text')
        text_vis = text_image.visualize(
            palette=['00000000'], # Transparent background
            forceRgbOutput=True
        ).drawText(
            text=str(yr),
            pos='top-center',
            fontSize=24,
            textColor='white',
            outlineColor='black',
            outlineWidth=2.5,
            outlineOpacity=0.7
        )

        # Visualize the data and mosaic the text on top
        visualized_img = img.visualize(**vis_params)
        frame = ee.ImageCollection([visualized_img, text_vis]).mosaic()
        frames.append(frame)

    col = ee.ImageCollection(frames)
    url = col.getVideoThumbURL({
        'dimensions': resolution_dd.value,
        'region': REGION,
        'framesPerSecond': 1,
        'format': 'gif',
    })
    
    resp = requests.get(url)
    county_name = f"_{county_dd.value}" if region_type_dd.value == 'County' else ''
    fn = f"anim_{p}_{state_dd.value}{county_name}_{start}_{end}.gif".replace(" ", "_")
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))


# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()

year_dd = widgets.Dropdown(options=list(range(1985, 2025)), value=2024, description='Year:')
param_dd = widgets.Dropdown(options=['LST', 'NDVI', 'NDBI', 'UTFVI', 'Hotspots'], value='LST', description='Parameter:')
state_dd = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State', 'County'], value='State', description='Region Type:')
county_dd = widgets.Dropdown(options=[], description='County:')
resolution_dd = widgets.IntText(value=512, description='Resolution(px):')
scale_dd = widgets.IntText(value=1000, description='Scale(m):')
download_map_btn = widgets.Button(description='Download Map')
anim_start_dd = widgets.Dropdown(options=list(range(1985, 2025)), value=2020, description='Anim Start:')
anim_end_dd = widgets.Dropdown(options=list(range(1985, 2025)), value=2024, description='Anim End:')
step_dd = widgets.IntSlider(value=1, min=1, max=10, description='Step(yrs):')
animate_btn = widgets.Button(description='Download Animation')
info_lbl = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

# To populate the county list on startup, simulate a "change" event.
update_counties(type('x', (), {'new': state_dd.value}))

# build UI layout
map_controls = widgets.VBox([
    widgets.HBox([param_dd, year_dd, resolution_dd, scale_dd]),
    widgets.HBox([state_dd, region_type_dd, county_dd])
])
download_controls = widgets.HBox([download_map_btn])
anim_controls = widgets.HBox([anim_start_dd, anim_end_dd, step_dd, animate_btn])
ui = widgets.VBox([map_controls, download_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map(center=[40, -98], zoom=4)
styled_counties = COUNTY_FC.style(**{
    'color': '0000FF',      # blue outline
    'fillColor': '00000000', # transparent fill
    'width': 1
})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
update_map()


Map(center=[40, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [5]:
pip install geemap ipyleaflet ipywidgets earthengine-api requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import ee
import geemap

In [7]:
ee.Initialize()  # Ensure Earth Engine is initialized
ee.Authenticate()  # Authenticate if needed

True

In [27]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC  = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info  = STATE_FC.getInfo()['features']
STATE_NAMES  = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts (no 'crs' keys)
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40,   'palette': ['blue','green','yellow','red']}
ndvi_vis    = {'min':   0, 'max':  1,   'palette': ['white','lightgreen','darkgreen']}
ndbi_vis    = {'min':  -1, 'max':  1,   'palette': ['brown','white','blue']}
utfvi_vis   = {'min': -0.5,'max': 0.5,  'palette': ['green','white','red']}
hotspot_vis = {'min':   0, 'max':  1,   'palette': ['lightgray','red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    if   1985 <= year <= 2011:
        coll  = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST':'ST_B6','NDVI':('SR_B4','SR_B3'),'NDBI':('SR_B5','SR_B4')}
    elif 2012 <= year <= 2013:
        coll  = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST':'ST_B6','NDVI':('SR_B4','SR_B3'),'NDBI':('SR_B5','SR_B4')}
    elif 2014 <= year <= 2022:
        coll  = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST':'ST_B10','NDVI':('SR_B5','SR_B4'),'NDBI':('SR_B6','SR_B5')}
    else:
        coll  = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST':'ST_B10','NDVI':('SR_B5','SR_B4'),'NDBI':('SR_B6','SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                     .multiply(0.00341802)
                     .add(149.0)
                     .subtract(273.15)
                     .rename('surface_temp'))
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i, True))
              .map(to_celsius)
              .median()
              .clip(region)
              .selfMask())

def get_ndvi(year, region):
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i))
              .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
              .median()
              .clip(region))

def get_ndbi(year, region):
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i))
              .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
              .median()
              .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    # remove old
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)
    # build new
    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    pal, st, mx = vis['palette'], vis['min'], vis['max']
    step = (mx - st) / len(pal)
    for i, col in enumerate(pal):
        lo, hi = st + i*step, st + (i+1)*step
        lbl = "≤{:.2f}".format(hi) if i==0 else (">{:.2f}".format(lo) if i==len(pal)-1 else "{:.2f}-{:.2f}".format(lo,hi))
        html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:{col};'></div>{lbl}</div>"
    html += "</div>"
    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips  = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    county_dd.value   = names[0]

# -----------------------------------------------------------------------------
# 9. Map update (clip to selected region)
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    global _prev_layer, REGION
    if _prev_layer:
        m.remove_layer(_prev_layer)

    # choose feature
    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value

    # compute image & vis
    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION),    ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION),    ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv  = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst   = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            REGION, scale_dd.value
        )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    # clip & add
    img = img.clip(REGION)
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"{p} {yr} (clipped to {region_type_dd.value})"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def download_map(b):
    yr, p = int(year_dd.value), param_dd.value
    img, vis = (get_annual_lst(yr, REGION), thermal_vis) if p=='LST' else (get_ndvi(yr, REGION), ndvi_vis)
    url = img.getThumbURL({
        'min': vis['min'], 'max': vis['max'],
        'palette': vis['palette'],
        'region': REGION,
        'dimensions': resolution_dd.value,
        'format': 'png'
    })
    resp = requests.get(url)
    fn   = f"map_{p}_{state_dd.value}_{region_type_dd.value}_" \
           f"{(county_dd.value if region_type_dd.value=='County' else state_dd.value)}_{yr}.png"
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

def download_animation(b):
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    yrs         = list(range(start, end+1, step_dd.value))
    frames      = []
    for yr in yrs:
        if param_dd.value=='LST':
            frames.append(get_annual_lst(yr, REGION).visualize(**thermal_vis))
        else:
            frames.append(get_ndvi(yr, REGION).visualize(**ndvi_vis))
    col = ee.ImageCollection(frames)
    url = col.getVideoThumbURL({
        'dimensions': resolution_dd.value,
        'region':      REGION,
        'framesPerSecond': 1,
        'format': 'gif',
        'backgroundColor': 'white'
    })
    resp = requests.get(url)
    fn   = f"anim_{param_dd.value}_{state_dd.value}_{region_type_dd.value}_" \
           f"{(county_dd.value if region_type_dd.value=='County' else state_dd.value)}_{start}_{end}.gif"
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()

year_dd        = widgets.Dropdown(options=list(range(1985,2025)), value=2025, description='Year:')
param_dd       = widgets.Dropdown(options=['LST','NDVI','NDBI','UTFVI','Hotspots'], value='LST', description='Parameter:')
state_dd       = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State','County'], value='State', description='Region Type:')
county_dd      = widgets.Dropdown(options=[], description='County:')
resolution_dd  = widgets.IntText(value=256, description='Resolution(px):')
scale_dd       = widgets.IntText(value=1000, description='Scale(m):')
download_map_btn = widgets.Button(description='Download Map')
anim_start_dd    = widgets.Dropdown(options=list(range(1985,2025)), value=1985, description='Anim Start:')
anim_end_dd      = widgets.Dropdown(options=list(range(1985,2025)), value=2025, description='Anim End:')
step_dd          = widgets.IntSlider( value=1, min=1, max=10, description='Step(yrs):')
animate_btn      = widgets.Button(description='Download Animation')
info_lbl         = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

# initialize county list
update_counties(type('x', (), {'new': state_dd.value}))

# build UI layout
map_controls  = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd, resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui            = widgets.VBox([map_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map()
styled_counties = COUNTY_FC.style(**{
    'color':     '0000FF',    # blue outline
    'fillColor': '00000000',  # transparent fill
    'width':     1
})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
_prev_layer = None
update_map()


TraitError: Invalid selection: value not found

In [17]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC  = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info  = STATE_FC.getInfo()['features']
STATE_NAMES  = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts (no 'crs' keys)
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40,   'palette': ['blue','green','yellow','red']}
ndvi_vis    = {'min':   0, 'max':  1,   'palette': ['white','lightgreen','darkgreen']}
ndbi_vis    = {'min':  -1, 'max':  1,   'palette': ['brown','white','blue']}
utfvi_vis   = {'min': -0.5,'max': 0.5,  'palette': ['green','white','red']}
hotspot_vis = {'min':   0, 'max':  1,   'palette': ['lightgray','red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    if   1985 <= year <= 2011:
        coll  = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST':'ST_B6','NDVI':('SR_B4','SR_B3'),'NDBI':('SR_B5','SR_B4')}
    elif 2012 <= year <= 2013:
        coll  = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST':'ST_B6','NDVI':('SR_B4','SR_B3'),'NDBI':('SR_B5','SR_B4')}
    elif 2014 <= year <= 2022:
        coll  = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST':'ST_B10','NDVI':('SR_B5','SR_B4'),'NDBI':('SR_B6','SR_B5')}
    else:
        coll  = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST':'ST_B10','NDVI':('SR_B5','SR_B4'),'NDBI':('SR_B6','SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                     .multiply(0.00341802)
                     .add(149.0)
                     .subtract(273.15)
                     .rename('surface_temp'))
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i, True))
              .map(to_celsius)
              .median()
              .clip(region)
              .selfMask())

def get_ndvi(year, region):
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i))
              .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
              .median()
              .clip(region))

def get_ndbi(year, region):
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i))
              .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
              .median()
              .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    # remove old
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)
    # build new
    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    pal, st, mx = vis['palette'], vis['min'], vis['max']
    step = (mx - st) / len(pal)
    for i, col in enumerate(pal):
        lo, hi = st + i*step, st + (i+1)*step
        lbl = "≤{:.2f}".format(hi) if i==0 else (">{:.2f}".format(lo) if i==len(pal)-1 else "{:.2f}-{:.2f}".format(lo,hi))
        html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:{col};'></div>{lbl}</div>"
    html += "</div>"
    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips  = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    county_dd.value   = names[0]

# -----------------------------------------------------------------------------
# 9. Map update (clip to selected region)
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    global _prev_layer, REGION
    if _prev_layer:
        m.remove_layer(_prev_layer)

    # choose feature
    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value

    # compute image & vis
    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION),    ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION),    ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv  = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst   = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            REGION, scale_dd.value
        )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    img = img.clip(REGION)
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"{p} {yr} (clipped to {region_type_dd.value})"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def download_map(b):
    yr, p = int(year_dd.value), param_dd.value
    img, vis = (get_annual_lst(yr, REGION), thermal_vis) if p=='LST' else (get_ndvi(yr, REGION), ndvi_vis)
    url = img.getThumbURL({
        'min': vis['min'], 'max': vis['max'],
        'palette': vis['palette'],
        'region': REGION,
        'dimensions': resolution_dd.value,
        'format': 'png'
    })
    resp = requests.get(url)
    fn   = f"map_{p}_{state_dd.value}_{region_type_dd.value}_" \
           f"{(county_dd.value if region_type_dd.value=='County' else state_dd.value)}_{yr}.png"
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

def download_animation(b):
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    yrs = list(range(start, end + 1, step_dd.value))
    frames = []

    for yr in yrs:
        if param_dd.value == 'LST':
            img = get_annual_lst(yr, REGION).visualize(**thermal_vis)
        else:
            img = get_ndvi(yr, REGION).visualize(**ndvi_vis)

        # Build dynamic title
        if region_type_dd.value == 'County':
            title_text = f"{param_dd.value} ({county_dd.value}) {yr}"
        else:
            title_text = f"{param_dd.value} ({state_dd.value}) {yr}"

        # Add title to image
        img_with_title = geemap.add_text_to_image(
            image=img,
            text=title_text,
            font_size=20,
            font_color='black',
            outline_color='white',
            outline_width=2,
            position='top'
        )

        frames.append(img_with_title)

    col = ee.ImageCollection(frames)
    url = col.getVideoThumbURL({
        'dimensions': resolution_dd.value,
        'region': REGION,
        'framesPerSecond': 1,
        'format': 'gif',
        'backgroundColor': 'white'
    })

    resp = requests.get(url)
    fn = f"anim_{param_dd.value}_{state_dd.value}_{region_type_dd.value}_" \
         f"{(county_dd.value if region_type_dd.value=='County' else state_dd.value)}_{start}_{end}.gif"
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()

year_dd        = widgets.Dropdown(options=list(range(1985,2025)), value=2024, description='Year:')
param_dd       = widgets.Dropdown(options=['LST','NDVI','NDBI','UTFVI','Hotspots'], value='LST', description='Parameter:')
state_dd       = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State','County'], value='State', description='Region Type:')
county_dd      = widgets.Dropdown(options=[], description='County:')
resolution_dd  = widgets.IntText(value=256, description='Resolution(px):')
scale_dd       = widgets.IntText(value=1000, description='Scale(m):')
download_map_btn = widgets.Button(description='Download Map')
anim_start_dd    = widgets.Dropdown(options=list(range(1985,2025)), value=1985, description='Anim Start:')
anim_end_dd      = widgets.Dropdown(options=list(range(1985,2025)), value=2024, description='Anim End:')
step_dd          = widgets.IntSlider( value=1, min=1, max=10, description='Step(yrs):')
animate_btn      = widgets.Button(description='Download Animation')
info_lbl         = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

update_counties(type('x', (), {'new': state_dd.value}))

map_controls  = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd, resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui            = widgets.VBox([map_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map()
styled_counties = COUNTY_FC.style(**{
    'color':     '0000FF',
    'fillColor': '00000000',
    'width':     1
})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
_prev_layer = None
update_map()


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

AttributeError: module 'geemap' has no attribute 'add_text_to_image'

In [33]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os
import datetime

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    """Authenticates and initializes the Earth Engine library."""
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info = STATE_FC.getInfo()['features']
STATE_NAMES = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40, 'palette': ['blue', 'green', 'yellow', 'red']}
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['white', 'lightgreen', 'darkgreen']}
ndbi_vis = {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'blue']}
utfvi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'red']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['lightgray', 'red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    """Selects the correct Landsat collection and bands based on the year."""
    if 1985 <= year <= 2011:
        coll = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2012 <= year <= 2013:
        coll = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2014 <= year <= 2022:
        coll = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    else:  # 2023 onwards
        coll = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    """Applies cloud, shadow, and optional radiometric saturation masks."""
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    """Computes the median annual Land Surface Temperature in Celsius."""
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                .multiply(0.00341802)
                .add(149.0)
                .subtract(273.15)
                .rename('surface_temp'))
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda i: apply_mask(i, True))
            .map(to_celsius)
            .median()
            .clip(region)
            .selfMask())

def get_ndvi(year, region):
    """Computes the median annual Normalized Difference Vegetation Index."""
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
            .median()
            .clip(region))

def get_ndbi(year, region):
    """Computes the median annual Normalized Difference Built-up Index."""
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
            .median()
            .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    """Removes old legends and adds a new one to the map."""
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)

    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    pal, st, mx = vis['palette'], vis['min'], vis['max']
    step = (mx - st) / len(pal)
    for i, col in enumerate(pal):
        lo, hi = st + i * step, st + (i + 1) * step
        lbl = "≤{:.2f}".format(hi) if i == 0 else (">{:.2f}".format(lo) if i == len(pal) - 1 else "{:.2f}-{:.2f}".format(lo, hi))
        html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:{col};'></div>{lbl}</div>"
    html += "</div>"

    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    """Updates the county dropdown based on the selected state."""
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    if names:
        county_dd.value = names[0]

# -----------------------------------------------------------------------------
# 9. Map update (clip to selected region)
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    """Main function to recompute and display the map layer."""
    global _prev_layer, REGION
    if _prev_layer:
        try:
            m.remove_layer(_prev_layer)
        except Exception:
            pass

    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        if not county_dd.value:
            info_lbl.value = f"No counties found for {state_dd.value}. Please select 'State' region type."
            return

        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value

    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION), ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            REGION, scale_dd.value
        )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    img = img.clip(REGION)
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"Displayed: {p} for {yr}"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def get_image_for_param(p, yr, region):
    """Helper to get the correct image and vis params for downloads."""
    if p == 'LST':
        return get_annual_lst(yr, region), thermal_vis
    elif p == 'NDVI':
        return get_ndvi(yr, region), ndvi_vis
    elif p == 'NDBI':
        return get_ndbi(yr, region), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, region)
        mv = lst.reduceRegion(ee.Reducer.mean(), region, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        return img, utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, region)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), region, scale_dd.value)
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        return img, hotspot_vis

def download_map(b):
    """Downloads the current map view as a PNG image."""
    info_lbl.value = "Generating map URL... Please wait."
    yr, p = int(year_dd.value), param_dd.value
    img, vis = get_image_for_param(p, yr, REGION)
    
    try:
        url = img.getThumbURL({
            'min': vis['min'], 'max': vis['max'], 'palette': vis['palette'],
            'region': REGION, 'dimensions': resolution_dd.value, 'format': 'png'
        })
        resp = requests.get(url)
        resp.raise_for_status()
        
        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"map_{p}_{state_dd.value}_{region_name}_{yr}.png"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = f"Success! Saved map as {fn}"
        display(FileLink(fn))
    except requests.exceptions.RequestException as e:
        info_lbl.value = f"Error: Download failed. Could not fetch image from server. {e}"
    except Exception as e:
        info_lbl.value = f"An unexpected error occurred: {e}"

def download_animation(b):
    """Generates and downloads a GIF animation for a selected time range."""
    info_lbl.value = "Generating animation... This may take several minutes."
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    p = param_dd.value
    
    last_full_year = CURRENT_YEAR - 1
    yrs = list(range(start, end + 1, step_dd.value))
    if last_full_year not in yrs:
        yrs.append(last_full_year) 
    yrs = sorted(list(set(yrs)))
    
    frames = []
    for yr in yrs:
        # MODIFIED: Text overlay functionality has been removed.
        img, vis = get_image_for_param(p, yr, REGION)
        visualized_img = img.visualize(**vis)
        frames.append(visualized_img)

    try:
        col = ee.ImageCollection(frames)
        url = col.getVideoThumbURL({
            'dimensions': resolution_dd.value, 'region': REGION,
            'framesPerSecond': 1, 'format': 'gif'
        })
        resp = requests.get(url)
        resp.raise_for_status()

        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"anim_{p}_{state_dd.value}_{region_name}_{start}_{end}.gif"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = f"Success! Saved animation as {fn}"
        display(FileLink(fn))
    except requests.exceptions.RequestException as e:
        info_lbl.value = f"Error: Download failed. Could not fetch GIF from server. {e}"
    except Exception as e:
        info_lbl.value = f"An unexpected error occurred: {e}"

# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()
CURRENT_YEAR = datetime.datetime.now().year
YEARS = list(range(1985, CURRENT_YEAR + 1))

param_dd = widgets.Dropdown(options=['LST', 'NDVI', 'NDBI', 'UTFVI', 'Hotspots'], value='LST', description='Parameter:')
year_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR, description='Year:')
state_dd = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State', 'County'], value='State', description='Region Type:')
county_dd = widgets.Dropdown(options=[], description='County:')
scale_dd = widgets.IntText(value=1000, description='Scale(m):', layout=widgets.Layout(width='200px'))
resolution_dd = widgets.IntText(value=512, description='Resolution(px):', layout=widgets.Layout(width='200px'))
download_map_btn = widgets.Button(description='Download Map')

anim_start_dd = widgets.Dropdown(options=YEARS, value=2000, description='Anim Start:')
anim_end_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR, description='Anim End:')
step_dd = widgets.IntSlider(value=2, min=1, max=10, description='Step(yrs):')
animate_btn = widgets.Button(description='Download Animation')
info_lbl = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

update_counties(type('x', (), {'new': state_dd.value}))

map_controls = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd])
download_controls = widgets.HBox([scale_dd, resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui = widgets.VBox([map_controls, download_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map(center=[40, -98], zoom=4)
styled_counties = COUNTY_FC.style(**{'color': '0000FF', 'fillColor': '00000000', 'width': 1})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
update_map()

Map(center=[40, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [34]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os
import datetime

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    """Authenticates and initializes the Earth Engine library."""
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info = STATE_FC.getInfo()['features']
STATE_NAMES = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40, 'palette': ['blue', 'green', 'yellow', 'red']}
# MODIFIED: A diverging brown-to-green palette.
# Brown = Low NDVI (barren/water), White = Mid/Soil, Green = High NDVI (vegetation)
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['#A52A2A', '#FFFFFF', '#006400']} # Brown, White, DarkGreen
ndbi_vis = {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'blue']}
utfvi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'red']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['lightgray', 'red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    """Selects the correct Landsat collection and bands based on the year."""
    if 1985 <= year <= 2011:
        coll = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2012 <= year <= 2013:
        coll = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2014 <= year <= 2022:
        coll = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    else:  # 2023 onwards
        coll = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    """Applies cloud, shadow, and optional radiometric saturation masks."""
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    """Computes the median annual Land Surface Temperature in Celsius."""
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                .multiply(0.00341802)
                .add(149.0)
                .subtract(273.15)
                .rename('surface_temp'))
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda i: apply_mask(i, True))
            .map(to_celsius)
            .median()
            .clip(region)
            .selfMask())

def get_ndvi(year, region):
    """Computes the median annual Normalized Difference Vegetation Index."""
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
            .median()
            .clip(region))

def get_ndbi(year, region):
    """Computes the median annual Normalized Difference Built-up Index."""
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
            .median()
            .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    """Removes old legends and adds a new one to the map."""
    # Remove any existing legend controls
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)

    pal = vis['palette']
    st = vis['min']
    mx = vis['max']

    # Create a continuous gradient legend for two-color palettes
    if len(pal) == 2:
        html = f"""
        <div style='padding: 8px; background: rgba(255,255,255,0.8); border-radius: 5px; width: 120px;'>
            <b style='display: block; margin-bottom: 5px;'>{title}</b>
            <div style='display: flex; align-items: center; justify-content: space-between;'>
                <span style='font-size: 11px;'>{st}</span>
                <span style='font-size: 11px;'>{mx}</span>
            </div>
            <div style='height: 15px; margin-top: 2px; border: 1px solid grey;
                        background: linear-gradient(to right, {pal[0]}, {pal[1]});'>
            </div>
        </div>
        """
    # Create a stepped legend for multi-color palettes
    else:
        html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);border-radius:5px;'><b>{title}</b><br>"
        step = (mx - st) / len(pal)
        for i, col in enumerate(pal):
            lo, hi = st + i * step, st + (i + 1) * step
            if i == 0:
                lbl = f"&le; {hi:.2f}"
            elif i == len(pal) - 1:
                lbl = f"&gt; {lo:.2f}"
            else:
                lbl = f"{lo:.2f} - {hi:.2f}"
            html += f"<div style='display:flex; align-items:center; margin-top:2px;'><div style='width:18px;height:12px;background:{col};border:1px solid grey;'></div>&nbsp;{lbl}</div>"
        html += "</div>"

    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    """Updates the county dropdown based on the selected state."""
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    if names:
        county_dd.value = names[0]

# -----------------------------------------------------------------------------
# 9. Map update (clip to selected region)
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    """Main function to recompute and display the map layer."""
    global _prev_layer, REGION
    if _prev_layer:
        try:
            m.remove_layer(_prev_layer)
        except Exception:
            pass

    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        if not county_dd.value:
            info_lbl.value = f"No counties found for {state_dd.value}. Please select 'State' region type."
            return

        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value

    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION), ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            REGION, scale_dd.value
        )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    img = img.clip(REGION)
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"Displayed: {p} for {yr}"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def get_image_for_param(p, yr, region):
    """Helper to get the correct image and vis params for downloads."""
    if p == 'LST':
        return get_annual_lst(yr, region), thermal_vis
    elif p == 'NDVI':
        return get_ndvi(yr, region), ndvi_vis
    elif p == 'NDBI':
        return get_ndbi(yr, region), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, region)
        mv = lst.reduceRegion(ee.Reducer.mean(), region, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        return img, utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, region)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), region, scale_dd.value)
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        return img, hotspot_vis

def download_map(b):
    """Downloads the current map view as a PNG image."""
    info_lbl.value = "Generating map URL... Please wait."
    yr, p = int(year_dd.value), param_dd.value
    img, vis = get_image_for_param(p, yr, REGION)
    
    try:
        url = img.getThumbURL({
            'min': vis['min'], 'max': vis['max'], 'palette': vis['palette'],
            'region': REGION, 'dimensions': resolution_dd.value, 'format': 'png'
        })
        resp = requests.get(url)
        resp.raise_for_status()
        
        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"map_{p}_{state_dd.value}_{region_name}_{yr}.png"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = f"Success! Saved map as {fn}"
        display(FileLink(fn))
    except requests.exceptions.RequestException as e:
        info_lbl.value = f"Error: Download failed. Could not fetch image from server. {e}"
    except Exception as e:
        info_lbl.value = f"An unexpected error occurred: {e}"

def download_animation(b):
    """Generates and downloads a GIF animation for a selected time range."""
    info_lbl.value = "Generating animation... This may take several minutes."
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    p = param_dd.value
    
    last_full_year = CURRENT_YEAR - 1
    yrs = list(range(start, end + 1, step_dd.value))
    if end < last_full_year and last_full_year not in yrs:
        yrs.append(last_full_year) 
    yrs = sorted(list(set(yrs)))
    
    frames = []
    for yr in yrs:
        img, vis = get_image_for_param(p, yr, REGION)
        visualized_img = img.visualize(**vis)
        frames.append(visualized_img)

    try:
        col = ee.ImageCollection(frames)
        url = col.getVideoThumbURL({
            'dimensions': resolution_dd.value, 'region': REGION,
            'framesPerSecond': 1, 'format': 'gif'
        })
        resp = requests.get(url)
        resp.raise_for_status()

        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"anim_{p}_{state_dd.value}_{region_name}_{start}_{end}.gif"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = f"Success! Saved animation as {fn}"
        display(FileLink(fn))
    except requests.exceptions.RequestException as e:
        info_lbl.value = f"Error: Download failed. Could not fetch GIF from server. {e}"
    except Exception as e:
        info_lbl.value = f"An unexpected error occurred: {e}"

# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()
CURRENT_YEAR = datetime.datetime.now().year
YEARS = list(range(1985, CURRENT_YEAR + 1))

param_dd = widgets.Dropdown(options=['LST', 'NDVI', 'NDBI', 'UTFVI', 'Hotspots'], value='LST', description='Parameter:')
year_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR - 1, description='Year:') # Default to last full year
state_dd = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State', 'County'], value='State', description='Region Type:')
county_dd = widgets.Dropdown(options=[], description='County:')
scale_dd = widgets.IntText(value=1000, description='Scale(m):', layout=widgets.Layout(width='200px'))
resolution_dd = widgets.IntText(value=512, description='Resolution(px):', layout=widgets.Layout(width='200px'))
download_map_btn = widgets.Button(description='Download Map')

anim_start_dd = widgets.Dropdown(options=YEARS, value=2000, description='Anim Start:')
anim_end_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR - 1, description='Anim End:') # Default to last full year
step_dd = widgets.IntSlider(value=2, min=1, max=10, description='Step(yrs):')
animate_btn = widgets.Button(description='Download Animation')
info_lbl = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

update_counties(type('x', (), {'new': state_dd.value}))

map_controls = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd])
download_controls = widgets.HBox([scale_dd, resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui = widgets.VBox([map_controls, download_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map(center=[40, -98], zoom=4)
styled_counties = COUNTY_FC.style(**{'color': '0000FF', 'fillColor': '00000000', 'width': 1})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
update_map()

Map(center=[40, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_Broward_1985_2024.gif

In [ ]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os
import datetime

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    """Authenticates and initializes the Earth Engine library."""
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info = STATE_FC.getInfo()['features']
STATE_NAMES = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40, 'palette': ['blue', 'green', 'yellow', 'red']}
# MODIFIED: Applying the 5-color palette from the user-provided image.
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['#8B0000', '#FF0000', '#FFFF00', '#00FF00', '#008000']}
ndbi_vis = {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'blue']}
utfvi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'red']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['lightgray', 'red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    """Selects the correct Landsat collection and bands based on the year."""
    if 1985 <= year <= 2011:
        coll = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2012 <= year <= 2013:
        coll = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2014 <= year <= 2022:
        coll = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    else:  # 2023 onwards
        coll = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    """Applies cloud, shadow, and optional radiometric saturation masks."""
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    """Computes the median annual Land Surface Temperature in Celsius."""
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                .multiply(0.00341802)
                .add(149.0)
                .subtract(273.15)
                .rename('surface_temp'))
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda i: apply_mask(i, True))
            .map(to_celsius)
            .median()
            .clip(region)
            .selfMask())

def get_ndvi(year, region):
    """Computes the median annual Normalized Difference Vegetation Index."""
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
            .median()
            .clip(region))

def get_ndbi(year, region):
    """Computes the median annual Normalized Difference Built-up Index."""
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
            .median()
            .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    """Removes old legends and adds a new one to the map."""
    # Remove any existing legend controls
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)

    pal = vis['palette']
    st = vis['min']
    mx = vis['max']

    # Create a continuous gradient legend for two-color palettes
    if len(pal) == 2:
        html = f"""
        <div style='padding: 8px; background: rgba(255,255,255,0.8); border-radius: 5px; width: 120px;'>
            <b style='display: block; margin-bottom: 5px;'>{title}</b>
            <div style='display: flex; align-items: center; justify-content: space-between;'>
                <span style='font-size: 11px;'>{st}</span>
                <span style='font-size: 11px;'>{mx}</span>
            </div>
            <div style='height: 15px; margin-top: 2px; border: 1px solid grey;
                        background: linear-gradient(to right, {pal[0]}, {pal[1]});'>
            </div>
        </div>
        """
    # Create a stepped legend for multi-color palettes
    else:
        html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);border-radius:5px;'><b>{title}</b><br>"
        step = (mx - st) / len(pal)
        for i, col in enumerate(pal):
            lo, hi = st + i * step, st + (i + 1) * step
            if i == 0:
                lbl = f"&le; {hi:.2f}"
            elif i == len(pal) - 1:
                lbl = f"&gt; {lo:.2f}"
            else:
                lbl = f"{lo:.2f} - {hi:.2f}"
            html += f"<div style='display:flex; align-items:center; margin-top:2px;'><div style='width:18px;height:12px;background:{col};border:1px solid grey;'></div>&nbsp;{lbl}</div>"
        html += "</div>"

    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    """Updates the county dropdown based on the selected state."""
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    if names:
        county_dd.value = names[0]

# -----------------------------------------------------------------------------
# 9. Map update (clip to selected region)
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    """Main function to recompute and display the map layer."""
    global _prev_layer, REGION
    if _prev_layer:
        try:
            m.remove_layer(_prev_layer)
        except Exception:
            pass

    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        if not county_dd.value:
            info_lbl.value = f"No counties found for {state_dd.value}. Please select 'State' region type."
            return

        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value

    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION), ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            REGION, scale_dd.value
        )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    img = img.clip(REGION)
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"Displayed: {p} for {yr}"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def get_image_for_param(p, yr, region):
    """Helper to get the correct image and vis params for downloads."""
    if p == 'LST':
        return get_annual_lst(yr, region), thermal_vis
    elif p == 'NDVI':
        return get_ndvi(yr, region), ndvi_vis
    elif p == 'NDBI':
        return get_ndbi(yr, region), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, region)
        mv = lst.reduceRegion(ee.Reducer.mean(), region, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        return img, utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, region)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), region, scale_dd.value)
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        return img, hotspot_vis

def download_map(b):
    """Downloads the current map view as a PNG image."""
    info_lbl.value = "Generating map URL... Please wait."
    yr, p = int(year_dd.value), param_dd.value
    img, vis = get_image_for_param(p, yr, REGION)
    
    try:
        url = img.getThumbURL({
            'min': vis['min'], 'max': vis['max'], 'palette': vis['palette'],
            'region': REGION, 'dimensions': resolution_dd.value, 'format': 'png'
        })
        resp = requests.get(url)
        resp.raise_for_status()
        
        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"map_{p}_{state_dd.value}_{region_name}_{yr}.png"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = f"Success! Saved map as {fn}"
        display(FileLink(fn))
    except requests.exceptions.RequestException as e:
        info_lbl.value = f"Error: Download failed. Could not fetch image from server. {e}"
    except Exception as e:
        info_lbl.value = f"An unexpected error occurred: {e}"

def download_animation(b):
    """Generates and downloads a GIF animation for a selected time range."""
    info_lbl.value = "Generating animation... This may take several minutes."
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    p = param_dd.value
    
    last_full_year = CURRENT_YEAR - 1
    yrs = list(range(start, end + 1, step_dd.value))
    if end < last_full_year and last_full_year not in yrs:
        yrs.append(last_full_year) 
    yrs = sorted(list(set(yrs)))
    
    frames = []
    for yr in yrs:
        img, vis = get_image_for_param(p, yr, REGION)
        visualized_img = img.visualize(**vis)
        frames.append(visualized_img)

    try:
        col = ee.ImageCollection(frames)
        url = col.getVideoThumbURL({
            'dimensions': resolution_dd.value, 'region': REGION,
            'framesPerSecond': 1, 'format': 'gif'
        })
        resp = requests.get(url)
        resp.raise_for_status()

        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"anim_{p}_{state_dd.value}_{region_name}_{start}_{end}.gif"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = f"Success! Saved animation as {fn}"
        display(FileLink(fn))
    except requests.exceptions.RequestException as e:
        info_lbl.value = f"Error: Download failed. Could not fetch GIF from server. {e}"
    except Exception as e:
        info_lbl.value = f"An unexpected error occurred: {e}"

# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()
CURRENT_YEAR = datetime.datetime.now().year
YEARS = list(range(1985, CURRENT_YEAR + 1))

param_dd = widgets.Dropdown(options=['LST', 'NDVI', 'NDBI', 'UTFVI', 'Hotspots'], value='LST', description='Parameter:')
year_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR - 1, description='Year:') # Default to last full year
state_dd = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State', 'County'], value='State', description='Region Type:')
county_dd = widgets.Dropdown(options=[], description='County:')
scale_dd = widgets.IntText(value=1000, description='Scale(m):', layout=widgets.Layout(width='200px'))
resolution_dd = widgets.IntText(value=512, description='Resolution(px):', layout=widgets.Layout(width='200px'))
download_map_btn = widgets.Button(description='Download Map')

anim_start_dd = widgets.Dropdown(options=YEARS, value=2000, description='Anim Start:')
anim_end_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR - 1, description='Anim End:') # Default to last full year
step_dd = widgets.IntSlider(value=2, min=1, max=10, description='Step(yrs):')
animate_btn = widgets.Button(description='Download Animation')
info_lbl = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

update_counties(type('x', (), {'new': state_dd.value}))

map_controls = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd])
download_controls = widgets.HBox([scale_dd, resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui = widgets.VBox([map_controls, download_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map(center=[40, -98], zoom=4)
styled_counties = COUNTY_FC.style(**{'color': '0000FF', 'fillColor': '00000000', 'width': 1})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
update_map()

Map(center=[40, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_Broward_1985_2024.gif

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_Charlotte_1985_2024.gif

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_DeSoto_1985_2024.gif

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_Hendry_1985_2024.gif

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_Highlands_1985_2024.gif

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_Jackson_1985_2024.gif

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_Miami-Dade_1985_2024.gif

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDVI_Florida_Orange_1985_2024.gif

c:\Users\ajayx\Desktop\OneDrive - Florida Atlantic University\RA\Conference\2025_GIS_Expo\geospatial\UHI-main\notebook\anim_NDBI_Florida_Florida_2000_2024.gif

In [40]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os
import datetime

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    """Authenticates and initializes the Earth Engine library."""
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info = STATE_FC.getInfo()['features']
STATE_NAMES = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40, 'palette': ['blue', 'green', 'yellow', 'red']}
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['#8B0000', '#FF0000', '#FFFF00', '#00FF00', '#008000']}
# MODIFIED: New palette for NDBI to show Built-up (Red), Soil (Yellow), and Vegetation (Green)
ndbi_vis = {'min': -1, 'max': 1, 'palette': ['#32CD32', '#FFD700', '#FF0000']} # Green, Yellow, Red
utfvi_vis = {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'red']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['lightgray', 'red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    """Selects the correct Landsat collection and bands based on the year."""
    if 1985 <= year <= 2011:
        coll = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2012 <= year <= 2013:
        coll = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2014 <= year <= 2022:
        coll = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    else:  # 2023 onwards
        coll = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    """Applies cloud, shadow, and optional radiometric saturation masks."""
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    """Computes the median annual Land Surface Temperature in Celsius."""
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                .multiply(0.00341802)
                .add(149.0)
                .subtract(273.15)
                .rename('surface_temp'))
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda i: apply_mask(i, True))
            .map(to_celsius)
            .median()
            .clip(region)
            .selfMask())

def get_ndvi(year, region):
    """Computes the median annual Normalized Difference Vegetation Index."""
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
            .median()
            .clip(region))

def get_ndbi(year, region):
    """Computes the median annual Normalized Difference Built-up Index."""
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
            .median()
            .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
# MODIFIED: Added a custom legend for NDBI based on user request.
def add_legend(vis, title):
    """Removes old legends and adds a new one to the map."""
    # Remove any existing legend controls
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)

    pal = vis['palette']
    st = vis['min']
    mx = vis['max']

    # Custom legend for NDBI
    if title == 'NDBI':
        html = """
        <div style='padding: 8px; background: rgba(255,255,255,0.8); border-radius: 5px; font-size: 12px;'>
            <b>NDBI</b><br>
            <div style='display: flex; align-items: center; margin-top: 4px;'>
                <div style='width: 20px; height: 15px; background: #FF0000; border: 1px solid grey;'></div>
                <span style='margin-left: 5px;'>&gt; 0.15 (Built-up)</span>
            </div>
            <div style='display: flex; align-items: center; margin-top: 4px;'>
                <div style='width: 20px; height: 15px; background: #FFD700; border: 1px solid grey;'></div>
                <span style='margin-left: 5px;'>~0 (Bare Soil)</span>
            </div>
            <div style='display: flex; align-items: center; margin-top: 4px;'>
                <div style='width: 20px; height: 15px; background: #32CD32; border: 1px solid grey;'></div>
                <span style='margin-left: 5px;'>&lt; 0 (Vegetation/Water)</span>
            </div>
        </div>
        """
    # Continuous gradient legend for two-color palettes
    elif len(pal) == 2:
        html = f"""
        <div style='padding: 8px; background: rgba(255,255,255,0.8); border-radius: 5px; width: 120px;'>
            <b style='display: block; margin-bottom: 5px;'>{title}</b>
            <div style='display: flex; align-items: center; justify-content: space-between;'>
                <span style='font-size: 11px;'>{st}</span>
                <span style='font-size: 11px;'>{mx}</span>
            </div>
            <div style='height: 15px; margin-top: 2px; border: 1px solid grey;
                        background: linear-gradient(to right, {pal[0]}, {pal[1]});'>
            </div>
        </div>
        """
    # Stepped legend for other multi-color palettes
    else:
        html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);border-radius:5px;'><b>{title}</b><br>"
        step = (mx - st) / len(pal)
        for i, col in enumerate(pal):
            lo, hi = st + i * step, st + (i + 1) * step
            if i == 0:
                lbl = f"&le; {hi:.2f}"
            elif i == len(pal) - 1:
                lbl = f"&gt; {lo:.2f}"
            else:
                lbl = f"{lo:.2f} - {hi:.2f}"
            html += f"<div style='display:flex; align-items:center; margin-top:2px;'><div style='width:18px;height:12px;background:{col};border:1px solid grey;'></div>&nbsp;{lbl}</div>"
        html += "</div>"

    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    """Updates the county dropdown based on the selected state."""
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    if names:
        county_dd.value = names[0]

# -----------------------------------------------------------------------------
# 9. Map update (clip to selected region)
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    """Main function to recompute and display the map layer."""
    global _prev_layer, REGION
    if _prev_layer:
        try:
            m.remove_layer(_prev_layer)
        except Exception:
            pass

    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        if not county_dd.value:
            info_lbl.value = f"No counties found for {state_dd.value}. Please select 'State' region type."
            return

        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value

    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION), ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            REGION, scale_dd.value
        )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    img = img.clip(REGION)
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"Displayed: {p} for {yr}"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def get_image_for_param(p, yr, region):
    """Helper to get the correct image and vis params for downloads."""
    if p == 'LST':
        return get_annual_lst(yr, region), thermal_vis
    elif p == 'NDVI':
        return get_ndvi(yr, region), ndvi_vis
    elif p == 'NDBI':
        return get_ndbi(yr, region), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, region)
        mv = lst.reduceRegion(ee.Reducer.mean(), region, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        return img, utfvi_vis
    else:  # Hotspots
        lst = get_annual_lst(yr, region)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), region, scale_dd.value)
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        return img, hotspot_vis

def download_map(b):
    """Downloads the current map view as a PNG image."""
    info_lbl.value = "Generating map URL... Please wait."
    yr, p = int(year_dd.value), param_dd.value
    img, vis = get_image_for_param(p, yr, REGION)
    
    try:
        url = img.getThumbURL({
            'min': vis['min'], 'max': vis['max'], 'palette': vis['palette'],
            'region': REGION, 'dimensions': resolution_dd.value, 'format': 'png'
        })
        resp = requests.get(url)
        resp.raise_for_status()
        
        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"map_{p}_{state_dd.value}_{region_name}_{yr}.png"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = f"Success! Saved map as {fn}"
        display(FileLink(fn))
    except requests.exceptions.RequestException as e:
        info_lbl.value = f"Error: Download failed. Could not fetch image from server. {e}"
    except Exception as e:
        info_lbl.value = f"An unexpected error occurred: {e}"

def download_animation(b):
    """Generates and downloads a GIF animation for a selected time range."""
    info_lbl.value = "Generating animation... This may take several minutes."
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    p = param_dd.value
    
    last_full_year = CURRENT_YEAR - 1
    yrs = list(range(start, end + 1, step_dd.value))
    if end < last_full_year and last_full_year not in yrs:
        yrs.append(last_full_year) 
    yrs = sorted(list(set(yrs)))
    
    frames = []
    for yr in yrs:
        img, vis = get_image_for_param(p, yr, REGION)
        visualized_img = img.visualize(**vis)
        frames.append(visualized_img)

    try:
        col = ee.ImageCollection(frames)
        url = col.getVideoThumbURL({
            'dimensions': resolution_dd.value, 'region': REGION,
            'framesPerSecond': 1, 'format': 'gif'
        })
        resp = requests.get(url)
        resp.raise_for_status()

        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"anim_{p}_{state_dd.value}_{region_name}_{start}_{end}.gif"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = f"Success! Saved animation as {fn}"
        display(FileLink(fn))
    except requests.exceptions.RequestException as e:
        info_lbl.value = f"Error: Download failed. Could not fetch GIF from server. {e}"
    except Exception as e:
        info_lbl.value = f"An unexpected error occurred: {e}"

# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()
CURRENT_YEAR = datetime.datetime.now().year
YEARS = list(range(1985, CURRENT_YEAR + 1))

param_dd = widgets.Dropdown(options=['LST', 'NDVI', 'NDBI', 'UTFVI', 'Hotspots'], value='LST', description='Parameter:')
year_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR - 1, description='Year:') # Default to last full year
state_dd = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State', 'County'], value='State', description='Region Type:')
county_dd = widgets.Dropdown(options=[], description='County:')
scale_dd = widgets.IntText(value=1000, description='Scale(m):', layout=widgets.Layout(width='200px'))
resolution_dd = widgets.IntText(value=512, description='Resolution(px):', layout=widgets.Layout(width='200px'))
download_map_btn = widgets.Button(description='Download Map')

anim_start_dd = widgets.Dropdown(options=YEARS, value=2000, description='Anim Start:')
anim_end_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR - 1, description='Anim End:') # Default to last full year
step_dd = widgets.IntSlider(value=2, min=1, max=10, description='Step(yrs):')
animate_btn = widgets.Button(description='Download Animation')
info_lbl = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

update_counties(type('x', (), {'new': state_dd.value}))

map_controls = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd])
download_controls = widgets.HBox([scale_dd, resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui = widgets.VBox([map_controls, download_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map(center=[40, -98], zoom=4)
styled_counties = COUNTY_FC.style(**{'color': '0000FF', 'fillColor': '00000000', 'width': 1})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
update_map()

Map(center=[40, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [ ]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os
import datetime

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    """Authenticates and initializes the Earth Engine library."""
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info = STATE_FC.getInfo()['features']
STATE_NAMES = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40, 'palette': ['blue', 'green', 'yellow', 'red']}
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['#8B0000', '#FF0000', '#FFFF00', '#00FF00', '#008000']}
ndbi_vis = {'min': -1, 'max': 1, 'palette': ['#32CD32', '#FFD700', '#FF0000']}

# MODIFIED: New palette for UTFVI using Green-White-Red color scheme.
# The min/max values correspond to the 6 discrete classes (0-5).
utfvi_vis = {
    'min': 0, 'max': 5,
    'palette': [
        '#006400',  # 0: Excellent (Dark Green)
        '#90EE90',  # 1: Good (Light Green)
        '#FFFFFF',  # 2: Normal (White)
        '#FFC0CB',  # 3: Bad (Pink / Light Red)
        '#FF0000',  # 4: Worse (Red)
        '#8B0000'   # 5: Worst (Dark Red)
    ]
}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['lightgray', 'red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    """Selects the correct Landsat collection and bands based on the year."""
    if 1985 <= year <= 2011:
        coll = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2012 <= year <= 2013:
        coll = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2014 <= year <= 2022:
        coll = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    else:  # 2023 onwards
        coll = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    """Applies cloud, shadow, and optional radiometric saturation masks."""
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    """Computes the median annual Land Surface Temperature in Celsius."""
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                .multiply(0.00341802)
                .add(149.0)
                .subtract(273.15)
                .rename('surface_temp'))
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda i: apply_mask(i, True))
            .map(to_celsius)
            .median()
            .clip(region)
            .selfMask())

def get_ndvi(year, region):
    """Computes the median annual Normalized Difference Vegetation Index."""
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
            .median()
            .clip(region))

def get_ndbi(year, region):
    """Computes the median annual Normalized Difference Built-up Index."""
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
            .filterBounds(region)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(apply_mask)
            .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
            .median()
            .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    """Removes old legends and adds a new one to the map."""
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)

    # MODIFIED: Updated legend colors for UTFVI to match new palette
    if title == 'UTFVI':
        html = """
        <div style='padding: 8px; background: rgba(255,255,255,0.8); border-radius: 5px; font-size: 12px;'>
            <b>UTFVI Category</b><br>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #8B0000; border: 1px solid grey;'></div><span style='margin-left: 5px;'>&gt; 0.02 (Worst)</span></div>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #FF0000; border: 1px solid grey;'></div><span style='margin-left: 5px;'>0.015 - 0.02 (Worse)</span></div>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #FFC0CB; border: 1px solid grey;'></div><span style='margin-left: 5px;'>0.01 - 0.015 (Bad)</span></div>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #FFFFFF; border: 1px solid grey;'></div><span style='margin-left: 5px;'>0.005 - 0.01 (Normal)</span></div>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #90EE90; border: 1px solid grey;'></div><span style='margin-left: 5px;'>0 - 0.005 (Good)</span></div>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #006400; border: 1px solid grey;'></div><span style='margin-left: 5px;'>&lt; 0 (Excellent)</span></div>
        </div>
        """
    # Custom legend for NDBI
    elif title == 'NDBI':
        html = """
        <div style='padding: 8px; background: rgba(255,255,255,0.8); border-radius: 5px; font-size: 12px;'>
            <b>NDBI</b><br>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #FF0000; border: 1px solid grey;'></div><span style='margin-left: 5px;'>&gt; 0.15 (Built-up)</span></div>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #FFD700; border: 1px solid grey;'></div><span style='margin-left: 5px;'>~0 (Bare Soil)</span></div>
            <div style='display: flex; align-items: center; margin-top: 4px;'><div style='width: 20px; height: 15px; background: #32CD32; border: 1px solid grey;'></div><span style='margin-left: 5px;'>&lt; 0 (Vegetation/Water)</span></div>
        </div>
        """
    # Generic legend for other palettes
    else:
        pal = vis['palette']
        st = vis['min']
        mx = vis['max']
        html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);border-radius:5px;'><b>{title}</b><br>"
        step = (mx - st) / len(pal)
        for i, col in enumerate(pal):
            lo, hi = st + i * step, st + (i + 1) * step
            lbl = f"{lo:.2f} - {hi:.2f}"
            html += f"<div style='display:flex; align-items:center; margin-top:2px;'><div style='width:18px;height:12px;background:{col};border:1px solid grey;'></div>&nbsp;{lbl}</div>"
        html += "</div>"

    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    """Updates the county dropdown based on the selected state."""
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    if names:
        county_dd.value = names[0]

# -----------------------------------------------------------------------------
# 9. Main map update and image fetching
# -----------------------------------------------------------------------------
_prev_layer = None

def get_image_for_param(p, yr, region):
    """Helper to get the correct image and vis params for display/downloads."""
    if p == 'LST':
        return get_annual_lst(yr, region), thermal_vis
    elif p == 'NDVI':
        return get_ndvi(yr, region), ndvi_vis
    elif p == 'NDBI':
        return get_ndbi(yr, region), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, region)
        mv = lst.reduceRegion(ee.Reducer.mean(), region, scale_dd.value).get('surface_temp')
        utfvi = lst.subtract(ee.Number(mv)).divide(ee.Number(mv))
        
        # Reclassify the continuous UTFVI image into 6 discrete classes (0-5)
        # This ensures the map colors correspond exactly to the defined ranges.
        classified_img = ee.Image(0).rename('UTFVI') \
            .where(utfvi.lt(0), 0) \
            .where(utfvi.gte(0).And(utfvi.lt(0.005)), 1) \
            .where(utfvi.gte(0.005).And(utfvi.lt(0.01)), 2) \
            .where(utfvi.gte(0.01).And(utfvi.lt(0.015)), 3) \
            .where(utfvi.gte(0.015).And(utfvi.lt(0.02)), 4) \
            .where(utfvi.gte(0.02), 5)
        
        return classified_img, utfvi_vis
    elif p == 'Hotspots':
        lst = get_annual_lst(yr, region)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), region, scale_dd.value)
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        return img, hotspot_vis
    else:
        raise ValueError("Invalid parameter selected.")

def update_map(*_):
    """Main function to recompute and display the map layer."""
    global _prev_layer, REGION
    if _prev_layer:
        try:
            m.remove_layer(_prev_layer)
        except Exception:
            pass

    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        if not county_dd.value:
            info_lbl.value = f"No counties found for {state_dd.value}. Please select 'State' region type."
            return
        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value
    
    try:
        img, vis = get_image_for_param(p, yr, REGION)
    except Exception as e:
        info_lbl.value = f"Error processing request: {e}"
        return

    _prev_layer = m.add_layer(img.clip(REGION), vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"Displayed: {p} for {yr}"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def download_map(b):
    """Downloads the current map view as a PNG image."""
    download_map_btn.disabled = True
    info_lbl.value = "Generating map URL... Please wait."
    
    try:
        yr, p = int(year_dd.value), param_dd.value
        img, vis = get_image_for_param(p, yr, REGION)
        
        url = img.getThumbURL({
            'min': vis['min'], 'max': vis['max'], 'palette': vis['palette'],
            'region': REGION, 'dimensions': resolution_dd.value, 'format': 'png'
        })
        resp = requests.get(url)
        resp.raise_for_status()
        
        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"map_{p}_{state_dd.value.replace(' ', '_')}_{region_name.replace(' ', '_')}_{yr}.png"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = "Success! Map saved."
        display(FileLink(fn))
    except Exception as e:
        info_lbl.value = f"Error: Download failed. {e}"
    finally:
        download_map_btn.disabled = False

def download_animation(b):
    """Generates and downloads a GIF animation for a selected time range."""
    animate_btn.disabled = True
    info_lbl.value = "Generating animation... This may take several minutes."
    
    try:
        start, end = int(anim_start_dd.value), int(anim_end_dd.value)
        p = param_dd.value
        yrs = list(range(start, end + 1, step_dd.value))
        
        frames = []
        for yr in yrs:
            img, vis = get_image_for_param(p, yr, REGION)
            visualized_img = img.visualize(**vis)
            frames.append(visualized_img)

        col = ee.ImageCollection(frames)
        url = col.getVideoThumbURL({
            'dimensions': resolution_dd.value, 'region': REGION,
            'framesPerSecond': 1, 'format': 'gif'
        })
        resp = requests.get(url)
        resp.raise_for_status()

        region_name = county_dd.value if region_type_dd.value == 'County' else state_dd.value
        fn = f"anim_{p}_{state_dd.value.replace(' ', '_')}_{region_name.replace(' ', '_')}_{start}_{end}.gif"
        
        with open(fn, 'wb') as f:
            f.write(resp.content)
        info_lbl.value = "Success! Animation saved."
        display(FileLink(fn))
    except Exception as e:
        info_lbl.value = f"Error: Animation failed. {e}"
    finally:
        animate_btn.disabled = False

# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()
CURRENT_YEAR = datetime.datetime.now().year
YEARS = list(range(1985, CURRENT_YEAR + 1))

param_dd = widgets.Dropdown(options=['LST', 'NDVI', 'NDBI', 'UTFVI', 'Hotspots'], value='LST', description='Parameter:')
year_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR - 1, description='Year:')
state_dd = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State', 'County'], value='State', description='Region Type:')
county_dd = widgets.Dropdown(options=[], description='County:')
scale_dd = widgets.IntText(value=1000, description='Scale(m):', layout=widgets.Layout(width='200px'))
resolution_dd = widgets.IntText(value=512, description='Resolution(px):', layout=widgets.Layout(width='200px'))
download_map_btn = widgets.Button(description='Download Map')

anim_start_dd = widgets.Dropdown(options=YEARS, value=2000, description='Anim Start:')
anim_end_dd = widgets.Dropdown(options=YEARS, value=CURRENT_YEAR - 1, description='Anim End:')
step_dd = widgets.IntSlider(value=2, min=1, max=10, description='Step(yrs):')
animate_btn = widgets.Button(description='Download Animation')
info_lbl = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

update_counties(type('x', (), {'new': state_dd.value}))

map_controls = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd])
download_controls = widgets.HBox([scale_dd, resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui = widgets.VBox([map_controls, download_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map(center=[40, -98], zoom=4)
styled_counties = COUNTY_FC.style(**{'color': '0000FF', 'fillColor': '00000000', 'width': 1})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
update_map()

Map(center=[40, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…